In [1]:
import re
import json
from pymongo import MongoClient

The file `trump.json` contains posts from http://www.twitter.com (tweets) over the course of an hour that have the key word "trump".
Each line in the file is a single JSON message that can be loaded with `json.loads()`.

In [7]:
#initialize the client,database, and collection
client = MongoClient()
db = client.db1
col = db.collection1
col.delete_many({})
#read in the info in the file and deserialize it while saving it in the collection
with open('trump.json','r') as file:
    fileinfo = file.readlines()
    for line in fileinfo:
        col.insert_one(json.loads(line))
#return the number of elements
col.count()

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  if __name__ == '__main__':


95643

In [78]:
russia = col.find({'text':{'$regex':re.compile('russia',re.IGNORECASE)}})
print('{} tweets contain the word Russia'.format(russia.count()))

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


5841 tweets contain the word Russia


In [79]:
#use regex to find the time zone names
CT = col.find({'user.time_zone':{'$regex':re.compile('Central')}})
PT = col.find({'user.time_zone':{'$regex':re.compile('Pacific')}})
ET = col.find({'user.time_zone':{'$regex':re.compile('Eastern')}})
MT = col.find({'user.time_zone':{'$regex':re.compile('Mountain')}})
print("{} tweets came from the main continental US time zones".format(CT.count()+PT.count()+ET.count()+MT.count()))


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """


34089 tweets came from the main continental US time zones


In [83]:
#make a list of all of the unique languages
languages = col.distinct('lang')
langs = {}
for language in languages:
    langs[language] = 0
#find how many tweets are contained in each language and save them
for x in langs:
    lang = col.find({'lang':x})
    langs[x] = lang.count()
#print results    
print('Frequency of languages:',langs)

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


Frequency of languages: {'en': 84919, 'fr': 1235, 'es': 3027, 'tr': 588, 'nl': 212, 'pt': 632, 'und': 1653, 'it': 423, 'de': 635, 'da': 35, 'zh': 1, 'ru': 29, 'pl': 108, 'no': 21, 'eu': 6, 'tl': 48, 'in': 121, 'cs': 13, 'ht': 21, 'ro': 21, 'sl': 3, 'sv': 55, 'fi': 20, 'ar': 30, 'et': 23, 'lt': 8, 'hu': 6, 'el': 10, 'ja': 17, 'lv': 2, 'vi': 7, 'fa': 5, 'ko': 2, 'ur': 1, 'cy': 2, 'th': 3, 'is': 1, 'iw': 1, 'uk': 3}


In [96]:
#find and sort from largest to smallest the users with the most followers
top5 = col.find({'user.followers_count':{'$exists':True}}).sort('user.followers_count',-1)
results = {}
#save the usernames of each of these guys without duplicates
for x in range(20):
    uname = top5[x]['user']['name']
    if uname not in results:
        results[uname] = top5[x]['user']['followers_count']
    #once we have all of the results that we want, quit
    if len(results) == 5:
        break
print(results)


{'CNN': 35481010, 'Reuters Top News': 17821353, 'Fox News': 14663497, 'Wall Street Journal': 14023091, 'TIME': 14017299}


In [140]:
hash5 = col.find({'entities.hashtags.4':{'$exists':True}}).sort('entities.hashtags.4.indices.0',1)[0]
print("The 5th hashtags appears earliest at character #{}".format(hash5['entities']['hashtags'][4]['indices'][0]))

The 5th hashtags appears earliest at character #23


In [109]:
top = col.find({'coordinates.coordinates':{'$exists':True}}).sort('coordinates.coordinates.0',-1)[0]
print(top['coordinates']['coordinates'])

[28.99237342, 41.06803023]


In [150]:
col.update_many({'retweeted_status':True},{'$unset':{'retweeted_status':True}})

In [151]:
col.update_many({'user.followers_count':{'$gte':1000}},{'$set':{'popular':True}})

In [153]:
col.find({"popular":True}).count()

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


32489